In [1]:
import random
from deap import base, creator, tools, algorithms

# Data karyawan: [gaji_per_jam, preferensi_jam_kerja (0: pagi, 1: siang, 2: malam)]
karyawan_data = [
    [20, 0],  # Karyawan 1: Gaji per jam $20, preferensi kerja pagi
    [25, 1],  # Karyawan 2: Gaji per jam $25, preferensi kerja siang
    [15, 2],  # Karyawan 3: Gaji per jam $15, preferensi kerja malam
    [30, 1],  # Karyawan 4: Gaji per jam $30, preferensi kerja siang
    [22, 0],  # Karyawan 5: Gaji per jam $22, preferensi kerja pagi
]

# Buat tipe fitness dengan dua tujuan (biaya dan kepuasan)
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))  # Minimasi biaya (-) dan maksimalkan kepuasan (+)
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()

# Representasi individu: satu individu mewakili penugasan karyawan (penempatan pada shift pagi, siang, malam)
toolbox.register("attr_int", random.randint, 0, 2)  # Shift kerja: 0 untuk pagi, 1 untuk siang, 2 untuk malam
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=len(karyawan_data))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Fungsi evaluasi multi-tujuan
def evalMultiObjective(individual):
    total_cost = 0  # Total biaya operasional
    total_satisfaction = 0  # Total kepuasan karyawan
    
    for i, shift in enumerate(individual):
        gaji_per_jam = karyawan_data[i][0]
        preferensi_kerja = karyawan_data[i][1]
        
        # Hitung biaya operasional
        total_cost += gaji_per_jam  # Setiap karyawan dikenakan biaya per jam yang tetap
        
        # Hitung kepuasan karyawan (0 = paling puas, -1 = kurang puas, -2 = tidak puas)
        if shift == preferensi_kerja:
            total_satisfaction += 1  # Tambah kepuasan jika sesuai preferensi
        elif abs(shift - preferensi_kerja) == 1:
            total_satisfaction += 0.5  # Tambah 0.5 jika tidak terlalu berbeda dari preferensi
        else:
            total_satisfaction -= 1  # Kurangi kepuasan jika shift berbeda jauh
        
    return total_cost, total_satisfaction

# Mendaftarkan operator-operator evolusi
toolbox.register("evaluate", evalMultiObjective)
toolbox.register("mate", tools.cxTwoPoint)  # Crossover dua titik
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Mutasi membalik bit (shift kerja)
toolbox.register("select", tools.selNSGA2)  # Seleksi NSGA-II

# Fungsi utama
def main():
    random.seed(42)
    pop = toolbox.population(n=50)  # Populasi 50 individu
    hof = tools.ParetoFront()  # Pareto Front untuk solusi optimal multi-tujuan
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", lambda x: sum([fit[0] for fit in x])/len(x))  # Rata-rata biaya
    stats.register("min", lambda x: min([fit[0] for fit in x]))  # Biaya minimum
    stats.register("max", lambda x: max([fit[1] for fit in x]))  # Kepuasan maksimum

    algorithms.eaMuPlusLambda(pop, toolbox, mu=50, lambda_=100, cxpb=0.7, mutpb=0.2, ngen=40,
                              stats=stats, halloffame=hof, verbose=True)

    return pop, hof, stats

if __name__ == "__main__":
    pop, hof, stats = main()

    # Tampilkan solusi Pareto optimal (biaya vs kepuasan)
    print("\nSolusi Pareto optimal (biaya operasional vs kepuasan karyawan):")
    for ind in hof:
        print(ind, "-> Biaya:", ind.fitness.values[0], "Kepuasan:", ind.fitness.values[1])


gen	nevals	avg	min	max
0  	50    	112	112	4  
1  	86    	112	112	4.5
2  	83    	112	112	4.5
3  	96    	112	112	5  
4  	88    	112	112	5  
5  	86    	112	112	5  
6  	93    	112	112	5  
7  	88    	112	112	5  
8  	87    	112	112	5  
9  	91    	112	112	5  
10 	92    	112	112	5  
11 	91    	112	112	5  
12 	89    	112	112	5  
13 	90    	112	112	5  
14 	90    	112	112	5  
15 	84    	112	112	5  
16 	90    	112	112	5  
17 	87    	112	112	5  
18 	90    	112	112	5  
19 	91    	112	112	5  
20 	86    	112	112	5  
21 	89    	112	112	5  
22 	87    	112	112	5  
23 	93    	112	112	5  
24 	86    	112	112	5  
25 	94    	112	112	5  
26 	94    	112	112	5  
27 	86    	112	112	5  
28 	93    	112	112	5  
29 	82    	112	112	5  
30 	90    	112	112	5  
31 	91    	112	112	5  
32 	90    	112	112	5  
33 	91    	112	112	5  
34 	94    	112	112	5  
35 	86    	112	112	5  
36 	89    	112	112	5  
37 	86    	112	112	5  
38 	88    	112	112	5  
39 	85    	112	112	5  
40 	87    	112	112	5  

Solusi Pareto optimal (biaya oper